In [ ]:
#SWN20140227 Table: 0
#remaining bug: bcf in the first column...

In [25]:
%load_ext autoreload
%autoreload 2
import pickle
import re
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from zlu_functions import del_subset_col, set_col_headers, \
                          row_header_to_col,remove_rows_for_total_value, \
                          dfs_need_transpose, remove_cols_for_fair_value, \
                          collar_fill_empty,remove_empty_rows_and_cols, \
                          df_column_uniquify, verticalSplit_instrument, \
                          verticalSplit_ZLU
from nel_functions import delete_uninformative,delete_empty,delete_duplicated, \
                          delete_subset,get_vertically_stacked_subtables, \
                          vertically_stacked_subtable_cleaner
from helper import volumeCleaner, dateCleaner, priceCleaner,trimDictonary, \
                   instrumentCleaner, productCleaner, filterBusiness,columnSorter, \
                   verticalSplit, anyCleaner
#pickle_in=open('../SEC_Data/Hedge/hedge_dfs_all_new.pickle','rb')

pickle_in=open('hedge_dfs_subset_8_18.pickle', 'rb')
df_dic=pickle.load(pickle_in)

delete_empty(df_dic)
df_dic = trimDictonary(df_dic,[])#remove empty list of dfs
del_dfs_dic_0 = delete_duplicated(df_dic) #remove duplicated dfs within a list; again
del_dfs_dic_1 = delete_subset(df_dic)

ticker_list = []

for k in df_dic.keys():
    if k[:3] not in ticker_list:
        print(k[:3])
        ticker_list.append(k[:3])


df_dic = filterBusiness(df_dic, 'CHK')

#ZLU change - if there is more than one table, 'december' indicates the summary table which do not want,
# and therefore we omit it.
for k in df_dic.keys():
    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
        df_col="".join([x.lower() for x in df.columns])
        if 'december' in df_col and len(df_dic[k])>1: df_dic[k][i]=pd.DataFrame()
        
delete_empty(df_dic)
#ZlU change

# df_dic={'CLR20100226':df_dic['CLR20100226']}
#del df_dic['PKD20090302'] #false positive
with open('output_before_spliting.csv','w') as f:
    pass
#preprocessing for all dfs
for k in df_dic.keys():
    for i in range(len(df_dic[k])):

        df=df_dic[k][i]
        df=set_col_headers(df) #set headers
        df=del_subset_col(df) #delete redundant cols
        df=remove_rows_for_total_value(df) #if any row for total value; save it
        df=remove_cols_for_fair_value(df)
        df_dic[k][i]=df
        with open('output_before_spliting.csv','a') as f:
            df=df_dic[k][i]
            f.writelines(k+'\n'+'table:'+str(i))
            try: df.to_csv(f,encoding='cp1252')
            except: df.to_csv(f,encoding='utf-8')

#         print(k,'Original Table:',i,'-'*10)
#         display(HTML(df.to_html()))

#Vertical Split
# for k in df_dic.keys(): #apply Sam's spliting function
    #print(k)
#     if 'RRC' in k: df_dic[k] = verticalSplit(df_dic[k])
#     elif k=='BRY20110301':
#         split_dfs = []
#         for table in df_dic[k]:
#             df1 = table.iloc[:, 0:3]
#             df2 = table.iloc[:, 3:]
#             split_dfs.append(df1)
#             split_dfs.append(df2)
#             df_dic[k]=split_dfs
#     else: df_dic[k] = verticalSplit(df_dic[k])
#     df_dic[k]=verticalSplit_instrument(df_dic[k])
#     if 'RRC' in k: df_dic[k] = verticalSplit(df_dic[k])
#     else: df_dic[k] = verticalSplit_ZLU(df_dic[k])
 

for k in df_dic.keys():
    #print(k)
    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
        if df.shape[1]==1: df=pd.DataFrame()
        elif isinstance(df, pd.DataFrame): #after new headers; ignore empty dfs
            df=del_subset_col(df) #delete redundant cols
            df=remove_rows_for_total_value(df) #if any row for total value; save it
            df=remove_cols_for_fair_value(df)
            df=row_header_to_col(df)
            df=dfs_need_transpose(df)
            df=collar_fill_empty(df)
            df=df_column_uniquify(df)
            df_dic[k][i]=remove_empty_rows_and_cols(df)
#             print(k,'Table:',i,'-'*10,'After preprocessing')
#             display(HTML(df.to_html()))


split_dictonary1 = {'oil_split' : ['oil'],
                    'gas_split' : ['gas'],
                    'ngl_split' : ['ngl']}

split_dictonary2 = {'fixed_split' : ['fixed'],
                    'call_split' : ['call'],
                    'diff_split' : ['differential']}

verbose_splits = False
#Vertical Split
for k in df_dic.keys(): #apply Sam's spliting function
    print(k)
    df_dic[k] = verticalSplit(df_dic[k], dictonary = split_dictonary1, verbose = verbose_splits)
    df_dic[k] = verticalSplit(df_dic[k], dictonary = split_dictonary2, verbose = verbose_splits)
#     df_dic[k] = verticalSplit_instrument(df_dic[k], kw1 = 'two way collar', kw2 = 'three way collar')
#     df_dic[k] = verticalSplit_instrument(df_dic[k], kw1 = 'two way collar', kw2 = 'fixed price')
#     df_dic[k] = verticalSplit_instrument(df_dic[k], kw1 = 'fixed price', kw2 = 'three way collar')
#     df_dic[k] = verticalSplit_instrument(df_dic[k], kw1 = 'basis swap', kw2 = 'three way collar')
#     df_dic[k] = verticalSplit_instrument(df_dic[k], kw1 = 'basis swap', kw2 = 'fixed price')

    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
        if df.shape[1]==1: df=pd.DataFrame()
        elif isinstance(df, pd.DataFrame): #after new headers; ignore empty dfs
            df=del_subset_col(df) #delete redundant cols
            df=remove_rows_for_total_value(df) #if any row for total value; save it
            df=remove_cols_for_fair_value(df)
            df=df_column_uniquify(df)
            df_dic[k][i]=remove_empty_rows_and_cols(df)
           
#Horizontally Split
del_dfs_dic3=get_vertically_stacked_subtables(df_dic)
vertically_stacked_subtable_cleaner(df_dic,del_dfs_dic3,exclusion=True, excluded_ticker=('MXC', 'OXY', 'HOC'))

delete_empty(df_dic)
df_dic = trimDictonary(df_dic,[])#remove empty list of dfs
del_dfs_dic_2 = delete_duplicated(df_dic) #remove duplicated dfs within a list; again
del_dfs_dic_3 = delete_subset(df_dic)

for k in df_dic.keys(): #apply Sam's spliting function
    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
        if df.shape[1]==1: df=pd.DataFrame()
        elif isinstance(df, pd.DataFrame): #after new headers; ignore empty dfs
            df=del_subset_col(df) #delete redundant cols
            df_dic[k][i]=df_column_uniquify(df)
            df_dic[k][i]=remove_empty_rows_and_cols(df)
            
with open('output_before_sam.csv','w') as f:
    pass
with open('output_after_sam.csv','w') as f:
    pass
for k in df_dic.keys():
    for i in range(len(df_dic[k])):
        with open('output_before_sam.csv','a') as f:
            df=df_dic[k][i]
            f.writelines(k+'\n'+'table:'+str(i))
            try: df.to_csv(f,encoding='cp1252')
            except: df.to_csv(f,encoding='utf-8')

date_ids = ["20\d{2}", "(short-term)|(long-term)"]             
date_stop = []
date_duplicate = True


price_keys = ['weighted average put', 'weighted average call', 'weighted average differential',
              'average strike price', 'fixedprice to bereceived (paid)', 'weighted average price,put,(per mmbtu)',
              'weighted average price,fixed,', 'weighted average price,call,(per mmbtu)',
              'weighted average price,differential', 'netpremiums,netpremiums,($ in millions)', 'weighted average price,put,(per bbl)', 
              'weighted average price,call,(per bbl)',
              ',weighted average price,call,($ per mmbtu)', ',weighted average price,put,($ per mmbtu)', ',weighted average price,call,($ per bbl)',
              ',weighted average price,put,($ per bbl)', 'fixed($ per mmbtu)', 'call($ per mmbtu)', 'differential($ per mmbtu)']

price_stop = []

volume_keys = ['bbtu', 'tbtu', 'mmbbl', 'volume']
volume_keys = [x.lower() for x in volume_keys]

volume_stop = []               
volume_helper = []

#set verbose to True to turn on the diagonstic output for Sam's functions
verbose = False
key_override = True
                
for k in df_dic.keys():
    print(k)
    for i in range(len(df_dic[k])):
        df=df_dic[k][i]
#         display(HTML(df.to_html()))
        if df.shape[0]>0: #apply Sam's functions to non-empty dfs
            print(k,'Table:',i,'-'*10,'Before Sam function')
            col_lst=[re.sub("[,]+","",col) if 'product_type' in col or 'instrument_type' in col else col for col in df.columns]
            df.columns=col_lst
            df = df.applymap(lambda s:s.lower() if type(s) == str else s)
            display(HTML(df.to_html()))
            df = instrumentCleaner(df, key_words = "", stop_words = "", verbose = verbose)
            df = productCleaner(df, key_words = "", stop_words = "", verbose = verbose)
            df = priceCleaner(df, key_words = price_keys, stop_words = price_stop,\
                                  verbose = verbose, key_override = key_override)
#             df = anyCleaner(df, key_words = volume_keys, stop_words = volume_stop, \
#                             units_column_helper = volume_helper, \
#                             key_override = key_override, condense = True, cleaner_label = 'vol')
#             display(HTML(df.to_html()))
            df = volumeCleaner(df, key_words = volume_keys, stop_words = volume_stop,\
                                   verbose = verbose, key_override = key_override)
            df = dateCleaner(df, value_ids = date_ids, impute = True, duplicate = date_duplicate)
            #ZLU's change begins
            display(HTML(df.to_html()))
            col_lst=[col for col in df.columns if 'vol' in col]
            for col in col_lst:
                df.loc[:,col]=df.loc[:,col].str.replace('[,\(\)]',"")
    #         display(HTML(df.to_html()))
    
    # build into functions volumeCleaner, row delete can stay here
#             if 'vol_2_type' in df.columns:
#                 for row_idx in range(df.shape[0]):
#                     if df.iloc[row_idx,df.columns=='vol_1'].values=="": 
#                         df.iloc[row_idx,df.columns=='vol_1'] = df.iloc[row_idx,df.columns=='vol_2'].values
#                         df.iloc[row_idx,df.columns=='vol_1_type']=df.iloc[row_idx,df.columns=='vol_2_type'].values
#                         df.iloc[row_idx,df.columns=='vol_2'] = ""
            col_lst=['vol' in col for col in df.columns]
            row_lst=list(range(df.shape[0]))
            for row_idx in range(df.shape[0]):
                if not any(df.iloc[row_idx,col_lst].str.contains('\d')):
                    row_lst.remove(row_idx)
            df=df.iloc[row_lst]
#             if all(df.loc[:,'vol_1_type']=='volume'): df.loc[:,'vol_1_type']=df.loc[:,'product_type'].str.extract('\(([^\)]+)\)')
#                         #ZLU's change ends
            print(k+' ,table:'+str(i)+'-----After Sam functions----')
            display(HTML(df.to_html()))
            print('~'*50)
        else: df=pd.DataFrame()
        df_dic[k][i]=df

del_dfs_dic_5 = delete_uninformative(df_dic)
print('1',len(df_dic))
delete_empty(df_dic)
print('2',len(df_dic))
df_dic = trimDictonary(df_dic,[])#remove empty list of dfs
print('3',len(df_dic))
del_dfs_dic_2 = delete_duplicated(df_dic) #remove duplicated dfs within a list; again
print('4',len(df_dic))
del_dfs_dic_3 = delete_subset(df_dic)
print('5',len(df_dic))

print(len(df_dic.keys()))

big_df = pd.DataFrame()

for k in df_dic.keys():
    doc_id = k
    processed_dfs = df_dic[k]
    for df in processed_dfs:
        unprocessed_cols = columnSorter(df)
        print(unprocessed_cols)
        df.drop(unprocessed_cols, axis = 1, inplace = True)
             #zlu change starts
        col_lst=['price' in col for col in df.columns]
        row_lst=list(range(df.shape[0]))
        df['red_flag']=''
        for row_idx in range(df.shape[0]):
            if not any(df.iloc[row_idx,col_lst].str.contains('\d')):
                df.iloc[row_idx,-1]=1
        #zlu change ends

#         display(HTML(df.to_html()))
        length = df.shape[0]
        doc_id_col = np.array([doc_id]*length)
        df.insert(0, 'doc_id', doc_id_col)
        big_df = pd.concat([big_df, df], axis = 0, join = 'outer', sort = False)

print(big_df.shape)

if any(big_df['red_flag'] != ''):
    print("Check the red_flag column for missing values at")
    #print('\n\n', big_df.iloc[big_df['red_flag'] != '', ].index)

try: big_df.to_csv('CHK_clean.csv', encoding='cp1252')
except: big_df.to_csv('CHK_clean.csv', encoding = 'utf-8')
# for k in df_dic.keys():
#     for i in range(len(df_dic[k])):
#         with open('APA_clean.csv','a') as f:
#             df=df_dic[k][i]
#             f.writelines(k+'\n'+'table:'+str(i))
#             try: df.to_csv(f,encoding='cp1252')
#             except: df.to_csv(f,encoding='utf-8')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
AHC
APA
APC
ARE
CHK
CLR
COG
COP
CPE
CRC
CRK
CRZ
CXO
DNR
DVN
ECA
EOG
GPO
HES
LPI
MRO
MTD
MUR
NBL
NOG
OAS
OXY
PDC
PE2
PET
PXD
QEP
RRC
SM2
SWN
UNT
WLL
WPX
WTI
XEC
CHK20030227
CHK20040315
CHK20050309
CHK20060314
CHK20070301
CHK20080229
CHK20090302
CHK20100301
CHK20110301
CHK20120229
CHK20130301
CHK20140227
CHK20150227
CHK20160225
CHK20030227
CHK20030227 Table: 0 ---------- Before Sam function


,,volume,average strike price,weighted average put strike price,weighted average differential to mid continent points,sfas 133 hedge,product_type,instrument_type
2,2003,57150000,4.31,,,yes,natural gas (mmbtu),swaps
4,2003,51100000,3.60,2.60,,no,natural gas (mmbtu),cap-swaps
6,2003,91250000,,,neg0.15,no,natural gas (mmbtu),basis protection swaps
7,2004,91500000,,,neg0.15,no,natural gas (mmbtu),basis protection swaps
8,2005,98550000,,,neg0.16,no,natural gas (mmbtu),basis protection swaps
9,2006,36500000,,,neg0.16,no,natural gas (mmbtu),basis protection swaps
10,2007,45625000,,,neg0.16,no,natural gas (mmbtu),basis protection swaps
11,2008,45750000,,,neg0.16,no,natural gas (mmbtu),basis protection swaps
12,2009,36500000,,,neg0.16,no,natural gas (mmbtu),basis protection swaps
14,2003,45700000,3.74,,,no,natural gas (mmbtu),counter-swaps


,maturity,vol_1,vol_1_type,price_1,price_1_type,price_2,price_2_type,price_3,price_3_type,sfas 133 hedge,product_type,instrument_type,old_maturity
2,2003,57150000,volume,4.31,average strike price,,weighted average put,,weighted average differential,yes,natural gas (mmbtu),swaps,2003
4,2003,51100000,volume,3.60,average strike price,2.60,weighted average put,,weighted average differential,no,natural gas (mmbtu),cap-swaps,2003
6,2003,91250000,volume,,average strike price,,weighted average put,neg0.15,weighted average differential,no,natural gas (mmbtu),basis protection swaps,2003
7,2004,91500000,volume,,average strike price,,weighted average put,neg0.15,weighted average differential,no,natural gas (mmbtu),basis protection swaps,2004
8,2005,98550000,volume,,average strike price,,weighted average put,neg0.16,weighted average differential,no,natural gas (mmbtu),basis protection swaps,2005
9,2006,36500000,volume,,average strike price,,weighted average put,neg0.16,weighted average differential,no,natural gas (mmbtu),basis protection swaps,2006
10,2007,45625000,volume,,average strike price,,weighted average put,neg0.16,weighted average differential,no,natural gas (mmbtu),basis protection swaps,2007
11,2008,45750000,volume,,average strike price,,weighted average put,neg0.16,weighted average differential,no,natural gas (mmbtu),basis protection swaps,2008
12,2009,36500000,volume,,average strike price,,weighted average put,neg0.16,weighted average differential,no,natural gas (mmbtu),basis protection swaps,2009
14,2003,45700000,volume,3.74,average strike price,,weighted average put,,weighted average differential,no,natural gas (mmbtu),counter-swaps,2003


CHK20030227 ,table:0-----After Sam functions----


,maturity,vol_1,vol_1_type,price_1,price_1_type,price_2,price_2_type,price_3,price_3_type,sfas 133 hedge,product_type,instrument_type,old_maturity
2,2003,57150000,volume,4.31,average strike price,,weighted average put,,weighted average differential,yes,natural gas (mmbtu),swaps,2003
4,2003,51100000,volume,3.60,average strike price,2.60,weighted average put,,weighted average differential,no,natural gas (mmbtu),cap-swaps,2003
6,2003,91250000,volume,,average strike price,,weighted average put,neg0.15,weighted average differential,no,natural gas (mmbtu),basis protection swaps,2003
7,2004,91500000,volume,,average strike price,,weighted average put,neg0.15,weighted average differential,no,natural gas (mmbtu),basis protection swaps,2004
8,2005,98550000,volume,,average strike price,,weighted average put,neg0.16,weighted average differential,no,natural gas (mmbtu),basis protection swaps,2005
9,2006,36500000,volume,,average strike price,,weighted average put,neg0.16,weighted average differential,no,natural gas (mmbtu),basis protection swaps,2006
10,2007,45625000,volume,,average strike price,,weighted average put,neg0.16,weighted average differential,no,natural gas (mmbtu),basis protection swaps,2007
11,2008,45750000,volume,,average strike price,,weighted average put,neg0.16,weighted average differential,no,natural gas (mmbtu),basis protection swaps,2008
12,2009,36500000,volume,,average strike price,,weighted average put,neg0.16,weighted average differential,no,natural gas (mmbtu),basis protection swaps,2009
14,2003,45700000,volume,3.74,average strike price,,weighted average put,,weighted average differential,no,natural gas (mmbtu),counter-swaps,2003


...

instrument_type column already exists


,maturity,vol_1,vol_1_type,price_1,price_1_type,product_type,instrument_type,old_maturity
24,short-term,33,tbtu,0.17,differential($ per mmbtu),natural gas,basis protection swaps,short-term
25,long-term,24,tbtu,neg0.48,differential($ per mmbtu),natural gas,basis protection swaps,long-term


CHK20160225 ,table:4-----After Sam functions----


,maturity,vol_1,vol_1_type,price_1,price_1_type,product_type,instrument_type,old_maturity
24,short-term,33,tbtu,0.17,differential($ per mmbtu),natural gas,basis protection swaps,short-term
25,long-term,24,tbtu,neg0.48,differential($ per mmbtu),natural gas,basis protection swaps,long-term


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
1 10
2 10
3 10
4 10
5 10
10
['sfas\xa0133 hedge']
['sfas 133hedge', 'premiumsreceived']
['sfas 133hedge']
['sfas 133hedge']
['sfas 133hedge']
['cash flowhedge,cash flowhedge,']
['cash flowhedge,cash flowhedge,']
['cash flowhedge,cash flowhedge,']
['cash flowhedge,cash flowhedge,']
['cash flowhedge,cash flowhedge,']
['cash flowhedge,cash flowhedge,']
['cash flowhedge,cash flowhedge,']
['cash flowhedge,cash flowhedge,']
['cash flowhedge,cash flowhedge,']
['cash flowhedge,cash flowhedge,']
['cash flowhedge,cash flowhedge,,']
['cash flowhedge,cash flowhedge,,']
['cash flowhedge,cash flowhedge,,']
['cash flowhedge,cash flowhedge,,']
[',designated,hedge,']
[',designated,hedge,']
[',designated,hedge,']
[',designated,hedge,']
[',designated,hedge,']
[',designated,hedge,']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
(251, 15)


In [60]:
lst = ['a']
type(lst[~0])

str

In [45]:
lst1 = [1, 2, 1, 0, 0]
lst2 = [0, 1, 1, 0, 2]

lst1 = [True if x == 0 else False for x in lst1]
lst2 = [True if x == 0 else False for x in lst2]

lst3 = [x & y for x,y in zip(lst1, lst2)]

print(lst1, '\n', lst2, '\n', lst3)

[False, False, False, True, True] 
 [True, False, False, True, False] 
 [False, False, False, True, False]


In [47]:
(big_df['red_flag'] == 1) & (big_df['red_flag'] != 1)

5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
5     False
6     False
7     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
      ...  
8     False
9     False
10    False
11    False
6     False
7     False
8     False
9     False
10    False
2     False
3     False
4     False
2     False
3     False
2     False
3     False
4     False
5     False
2     False
3     False
4     False
2     False
3     False
2     False
2     False
3     False
2     False
2     False
3     False
2     False
Name: red_flag, Length: 112, dtype: bool

In [48]:
not(True)

False